# Collecting and Organizing Data

In [20]:
import csv
import os
import pandas as pd
from pathlib import Path

First we have to make a master dataframe with all the relevant data. This master dataframe will contain an entry for every single onset, for every single wav file in the audio file. If an audio file is multiple drum sounds, then there is a single onset for each drum sound, and an single audio file will contirnbute to multiple entries in the dataset. We will have to parse AVP and LVT seperately and then combine them. 

In [21]:
# function to parse AVP csv, get the onset time, instrument label, onset phoneme, coda phoneme, dataset, participant id, subset, csv file path, wav file path
def parse_avp_csv(csv_path):
    """
    Parses an AVP CSV with no header, returning a list of dicts.
    Each dict has:
      - onset_time (float)
      - instrument_label (str)
      - onset_phoneme (str)
      - coda_phoneme (str)
      - dataset (str) = "AVP"
      - participant_id (str)
      - subset (str) = "personal"
      - csv_file_path (str)
      - wav_file_path (str)
    """
    # 1) Extract some metadata from the file path
    csv_dir = os.path.dirname(csv_path)             # e.g., "AVP_Dataset/Personal/Participant_1"
    csv_file_name = os.path.basename(csv_path)      # e.g., "P1_kd.csv" or "Participant_1_kd.csv"
    base_name, _ = os.path.splitext(csv_file_name)  # e.g., "P1_kd" or "Participant_1_kd"
    
    # 2) Determine participant_id from the file name
    #    Suppose your file name is "P1_kd.csv", so the participant part is "P1".
    #    If it’s "Participant_1_kd.csv", you might want the first two segments. Adapt as needed.
    #    Here's a simple approach:
    parts = base_name.split("_")  # e.g. ["P1", "kd"] or ["Participant", "1", "kd"]
    participant_id = parts[0]     # e.g. "P1" or "Participant"
    # If your actual naming is more complicated, tweak the logic. 
    # For instance, if "Participant_1" is always 2 segments, you might do participant_id = "_".join(parts[:2])
    
    # 3) Build the wav path. If the CSV is "P1_kd.csv", the wav is "P1_kd.wav" in the same folder.
    wav_file_name = base_name + ".wav"
    wav_file_path = os.path.join(csv_dir, wav_file_name)
    
    # 4) We'll fix "dataset" = "AVP" and "subset" = "personal" 
    #    (since that's the arrangement you described for the AVP dataset).
    dataset = "AVP"
    subset = "personal"
    
    # 5) Parse each row of the CSV
    data = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            # Expecting: [onset_time, instrument_label, onset_phoneme, coda_phoneme]
            if len(row) < 2:
                continue  # skip empty or malformed lines
            onset_time = float(row[0])
            instrument_label = row[1]
            onset_phoneme = row[2] if len(row) > 2 else ''
            coda_phoneme = row[3] if len(row) > 3 else ''
            
            entry = {
                'onset_time': onset_time,
                'instrument_label': instrument_label,
                'onset_phoneme': onset_phoneme,
                'coda_phoneme': coda_phoneme,
                'dataset': dataset,
                'participant_id': participant_id,
                'subset': subset,
                'csv_file_path': csv_path,
                'wav_file_path': wav_file_path
            }
            data.append(entry)
    
    return data

In [22]:
#function to collect all AVP data, takes a root directory as input, walks through the AVP dataset directory and collects all CSV data into a master DataFrame
def collect_all_avp_data(root_dir):
    """
    Walks through the AVP dataset directory and collects all CSV data into a master DataFrame,
    maintaining the grouping of entries from the same CSV file
    """
    all_data = []
    personal_dir = os.path.join(root_dir, "Personal")
    
    # Walk through all participant directories in sorted order
    for participant_dir in sorted(os.listdir(personal_dir)):
        participant_path = os.path.join(personal_dir, participant_dir)
        
        # Skip if not a directory or hidden files
        if not os.path.isdir(participant_path) or participant_dir.startswith('.'):
            continue
            
        # Process CSV files in sorted order
        for file_name in sorted(os.listdir(participant_path)):
            if file_name.endswith('.csv'):
                csv_path = os.path.join(participant_path, file_name)
                
                try:
                    parsed_data = parse_avp_csv(csv_path)
                    # Add the source filename as a field for sorting
                    for entry in parsed_data:
                        entry['source_file'] = file_name
                    all_data.extend(parsed_data)
                except Exception as e:
                    print(f"Error processing {csv_path}: {str(e)}")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort to maintain grouping:
    # First by participant_id, then by source file, then by onset_time
    df = df.sort_values(['participant_id', 'source_file', 'onset_time'])
    
    # Optionally remove the temporary source_file column if you don't need it
    df = df.drop('source_file', axis=1)
    
    return df

In [23]:
# def get_standardized_phoneme(simple_phoneme, is_onset=True):
#     """
#     Converts LVT phoneme notation to match AVP's IPA notation
    
#     Args:
#         simple_phoneme: The phoneme from LVT dataset
#         is_onset: Boolean indicating if this is an onset (True) or coda (False) phoneme
#     """
#     # Onset phonemes (consonants at start)
#     onset_map = {
#         '!': '!',        # keep as is (appears in both)
#         'k': 'k',        # keep as is
#         'p': 'p',        # keep as is
#         's': 's',        # keep as is
#         't': 't',        # keep as is
#         'ts': 'ts',      # keep as is
#         'tʃ': 'tʃ',      # keep as is
#         'ʔ': 'tʃ',       # map glottal stop to tʃ
#         'ʡʢ': 'ʡʢ'       # keep as is (appears in both)
#     }
    
#     # Coda phonemes (vowels/endings)
#     coda_map = {
#         'a': 'æ',        # map 'a' to 'æ' as it's more common in AVP
#         'h': 'h',        # keep as is
#         'u': 'u',        # keep as is
#         'x': 'x',        # keep as is
#         'ʊ': 'ʊ'         # keep as is
#     }
    
#     # Choose which mapping to use
#     phoneme_map = onset_map if is_onset else coda_map
    
#     # Return mapped phoneme if it exists, otherwise return original
#     return phoneme_map.get(simple_phoneme, simple_phoneme)

In [24]:
#functiuon to parse LVT csv, get the onset time, instrument label, onset phoneme, coda phoneme, dataset, participant id, subset, csv file path, wav file path
def parse_lvt_csv(csv_path):
    """
    Parses an LVT CSV with no header, returning a list of dicts.
    Similar to parse_avp_csv but handles LVT-specific formatting.
    """
    # Extract metadata from the file path
    csv_dir = os.path.dirname(csv_path)             
    csv_file_name = os.path.basename(csv_path)      # e.g., "AFRP.csv"
    base_name, _ = os.path.splitext(csv_file_name)  # e.g., "AFRP"
    
    # Determine if this is from Frase or Improviso folder
    subset = "Frase" if "Frase" in csv_dir else "Improviso"
    
    # Participant ID is the filename without extension
    participant_id = base_name
    
    # Build the wav path (add "3" before .wav)
    wav_file_name = base_name + "3.wav"
    wav_file_path = os.path.join(csv_dir, wav_file_name)
    
    # Mapping for instrument labels
    instrument_map = {
        "Kick": "kd",
        "Snare": "sd",
        "HH": "hhc"  # Assuming all HH in LVT are closed hi-hats
    }
    
    data = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) < 2:
                continue  # skip empty or malformed lines
                
            onset_time = float(row[0])
            original_label = row[1]
            instrument_label = instrument_map.get(original_label, original_label)
            onset_phoneme = row[2] if len(row) > 2 else ''
            coda_phoneme = row[3] if len(row) > 3 else ''
            
            # onset_phoneme = get_standardized_phoneme(row[2], is_onset=True)   # converts 'ts' if needed
            # coda_phoneme = get_standardized_phoneme(row[3], is_onset=False)   # converts 'x' if needed
            
            entry = {
                'onset_time': onset_time,
                'instrument_label': instrument_label,
                'onset_phoneme': onset_phoneme,
                'coda_phoneme': coda_phoneme,
                'dataset': "LVT",
                'participant_id': participant_id,
                'subset': subset,
                'csv_file_path': csv_path,
                'wav_file_path': wav_file_path
            }
            data.append(entry)
    
    return data

In [25]:
#function to collect all LVT data, takes a root directory as input, walks through the LVT dataset directory and collects all CSV data into a master DataFrame
def collect_all_lvt_data(root_dir):
    """
    Walks through the LVT dataset directory and collects all CSV data into a master DataFrame
    """
    all_data = []
    
    # Process both Frase and Improviso folders
    for subset_dir in ["Frase", "Improviso"]:
        subset_path = os.path.join(root_dir, subset_dir)
        
        # Skip if directory doesn't exist
        if not os.path.isdir(subset_path):
            continue
            
        # Process CSV files in sorted order
        for file_name in sorted(os.listdir(subset_path)):
            if file_name.endswith('.csv') and not file_name.startswith('.'):
                csv_path = os.path.join(subset_path, file_name)
                
                try:
                    parsed_data = parse_lvt_csv(csv_path)
                    # Add source file for sorting
                    for entry in parsed_data:
                        entry['source_file'] = file_name
                    all_data.extend(parsed_data)
                except Exception as e:
                    print(f"Error processing {csv_path}: {str(e)}")
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    
    # Sort to maintain grouping
    df = df.sort_values(['subset', 'participant_id', 'source_file', 'onset_time'])
    
    # Remove temporary sorting column
    df = df.drop('source_file', axis=1)
    
    return df

In [26]:
#function to create all datasets, calls the functions to collect AVP and LVT data and then combines them into a master dataframe
def create_all_datasets():
    avp_dataset_path = "../AVP-LVT_Dataset/AVP_Dataset"
    lvt_dataset_path = "../AVP-LVT_Dataset/LVT_Dataset"
    
    # Collect data from both datasets
    print("Processing AVP dataset...")
    avp_df = collect_all_avp_data(avp_dataset_path)
    
    print("Processing LVT dataset...")
    lvt_df = collect_all_lvt_data(lvt_dataset_path)
    
    # Save individual datasets
    print("\nSaving individual datasets...")
    # Ensure the directory exists
    os.makedirs('EDA', exist_ok=True)
    
    avp_df.to_csv('EDA/avp_dataset.csv', index=False)
    lvt_df.to_csv('EDA/lvt_dataset.csv', index=False)
    
    # Combine and save master dataset
    print("Creating and saving master dataset...")
    master_df = pd.concat([avp_df, lvt_df], ignore_index=True)
    master_df.to_csv('EDA/master_dataset.csv', index=False)
    
    # Print summary statistics
    print("\nDataset Summaries:")
    print(f"AVP Dataset: {len(avp_df)} events")
    print("\nAVP participants:", len(avp_df['participant_id'].unique()))
    print("AVP instrument distribution:")
    print(avp_df['instrument_label'].value_counts())
    
    print(f"\nLVT Dataset: {len(lvt_df)} events")
    print("LVT subsets:", lvt_df['subset'].unique())
    print("LVT participants:", len(lvt_df['participant_id'].unique()))
    print("LVT instrument distribution:")
    print(lvt_df['instrument_label'].value_counts())
    
    print(f"\nMaster Dataset: {len(master_df)} total events")
    print("Distribution by dataset:")
    print(master_df['dataset'].value_counts())
    
    return avp_df, lvt_df, master_df


In [27]:
avp_df, lvt_df, master_df = create_all_datasets()

Processing AVP dataset...
Processing LVT dataset...

Saving individual datasets...
Creating and saving master dataset...

Dataset Summaries:
AVP Dataset: 4873 events

AVP participants: 28
AVP instrument distribution:
instrument_label
kd     1447
sd     1253
hhc    1164
hho    1009
Name: count, dtype: int64

LVT Dataset: 841 events
LVT subsets: ['Frase' 'Improviso']
LVT participants: 40
LVT instrument distribution:
instrument_label
hhc    334
kd     329
sd     178
Name: count, dtype: int64

Master Dataset: 5714 total events
Distribution by dataset:
dataset
AVP    4873
LVT     841
Name: count, dtype: int64


Now we have master_dataset.csv, and master_df, both of which contain the info for every single onset for every single sound in the dataset. 

In [28]:
def analyze_phonemes():
    """
    Analyze and compare phonemes between AVP and LVT datasets
    """
    # Read both datasets
    avp_df = pd.read_csv('EDA/avp_dataset.csv')
    lvt_df = pd.read_csv('EDA/lvt_dataset.csv')
    
    print("AVP Unique Onset Phonemes:")
    print(sorted(avp_df['onset_phoneme'].unique()))
    print("\nAVP Unique Coda Phonemes:")
    print(sorted(avp_df['coda_phoneme'].unique()))
    
    print("\nLVT Unique Onset Phonemes:")
    print(sorted(lvt_df['onset_phoneme'].unique()))
    print("\nLVT Unique Coda Phonemes:")
    print(sorted(lvt_df['coda_phoneme'].unique()))

# Run the analysis
analyze_phonemes()

AVP Unique Onset Phonemes:
['!', 'dʒ', 'k', 'kg', 'kʃ', 'p', 's', 't', 'ts', 'tɕ', 'tʃ', 'tʒ', 'ʡʢ']

AVP Unique Coda Phonemes:
['I', 'a', 'e', 'h', 'i', 'o', 'u', 'x', 'æ', 'œ', 'ɐ', 'ɘ', 'ə', 'ɪ', 'ɯ', 'ʊ', 'ʌ']

LVT Unique Onset Phonemes:
['!', 'k', 'p', 's', 't', 'ts', 'tʃ', 'ʔ', 'ʡʢ']

LVT Unique Coda Phonemes:
['a', 'h', 'u', 'x', 'ʊ']


# Audio Segmentation

Using the master dataset's onset times, cut each continuous audio recording into individual "boxemes" (isolated vocal percussion sounds).

In [29]:
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from pathlib import Path
from tqdm import tqdm

def segment_audio(master_df, output_dir, segment_duration=0.5):
    """
    Segments audio files and saves them to disk.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    segment_info = []
    
    # Group by wav_file_path
    for wav_path, group in tqdm(master_df.groupby('wav_file_path')):
        print(f"\nProcessing {wav_path}")
        
        try:
            # Use soundfile instead of librosa.load
            y, sr = sf.read(wav_path)
            
            # Process each onset in this file
            for idx, row in group.iterrows():
                start_sample = int(row['onset_time'] * sr)
                end_sample = start_sample + int(segment_duration * sr)
                
                # Handle edge cases
                if start_sample < 0:
                    start_sample = 0
                if end_sample > len(y):
                    end_sample = len(y)
                
                if end_sample > start_sample:
                    segment = y[start_sample:end_sample]
                    
                    # Pad if needed
                    if len(segment) < int(segment_duration * sr):
                        segment = np.pad(segment, 
                                      (0, int(segment_duration * sr) - len(segment)),
                                      mode='constant')
                    
                    segment_filename = (f"{row['dataset']}_{row['participant_id']}_"
                                     f"{row['instrument_label']}_{idx:04d}.wav")
                    
                    segment_path = output_dir / segment_filename
                    sf.write(str(segment_path), segment, sr)
                    
                    segment_info.append({
                        'segment_path': str(segment_path),
                        'instrument_label': row['instrument_label'],
                        'participant_id': row['participant_id'],
                        'dataset': row['dataset'],
                        'original_wav': wav_path,
                        'onset_time': row['onset_time']
                    })
        
        except Exception as e:
            print(f"Error processing {wav_path}: {str(e)}")
            continue
    
    segment_df = pd.DataFrame(segment_info)
    segment_df.to_csv('EDA/segment_info.csv', index=False)
    
    return segment_df
    

In [ ]:
master_df = pd.read_csv('EDA/master_dataset.csv')
    
# Segment all audio files
print("Starting audio segmentation...")
segment_df = segment_audio(master_df, output_dir='segments')
    
# Print summary
print("\nSegmentation Summary:")
print(f"Total segments extracted: {len(segment_df)}")
print("\nInstrument distribution:")
print(segment_df['instrument_label'].value_counts())
print("\nDataset distribution:")
print(segment_df['dataset'].value_counts())

# Augment Segments via techniques in paper

In [21]:
import os
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from pathlib import Path
from tqdm import tqdm

def augment_segments(input_dir='segments', output_dir='augmentedSegments', num_augmentations=5):
    """
    Augment audio segments using pitch shifting and time stretching.
    
    Args:
        input_dir (str): Directory containing original segments
        output_dir (str): Directory to save augmented segments
        num_augmentations (int): Number of augmented versions to create per segment
    """
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True, parents=True)
    
    # Load original segment info
    original_info = pd.read_csv('EDA/segment_info.csv')
    augmented_info = []
    
    # Copy original segments and their info
    print("Copying original segments...")
    for _, row in tqdm(original_info.iterrows()):
        orig_path = Path(row['segment_path'])
        new_path = output_path / orig_path.name
        
        # Copy the audio file
        y, sr = sf.read(orig_path)
        sf.write(str(new_path), y, sr)
        
        # Add original file info to augmented dataset
        augmented_info.append({
            'segment_path': str(new_path),
            'instrument_label': row['instrument_label'],
            'participant_id': row['participant_id'],
            'dataset': row['dataset'],
            'original_wav': row['original_wav'],
            'onset_time': row['onset_time']
        })
    
    # Parameters for augmentation
    pitch_shift_range = (-1.5, 1.5)  # semitones
    time_stretch_range = (0.8, 1.2)   # rate
    
    # Process each file
    print("\nGenerating augmented segments...")
    for _, row in tqdm(original_info.iterrows()):
        orig_path = Path(row['segment_path'])
        y, sr = librosa.load(orig_path)
        
        # Create multiple augmented versions
        for i in range(num_augmentations):
            # Randomly choose augmentation order
            if np.random.random() > 0.5:
                # Pitch shift then time stretch
                pitch_shift = np.random.uniform(*pitch_shift_range)
                y_aug = librosa.effects.pitch_shift(y, sr=sr, n_steps=pitch_shift)
                
                time_stretch = np.random.uniform(*time_stretch_range)
                y_aug = librosa.effects.time_stretch(y_aug, rate=time_stretch)
            else:
                # Time stretch then pitch shift
                time_stretch = np.random.uniform(*time_stretch_range)
                y_aug = librosa.effects.time_stretch(y, rate=time_stretch)
                
                pitch_shift = np.random.uniform(*pitch_shift_range)
                y_aug = librosa.effects.pitch_shift(y_aug, sr=sr, n_steps=pitch_shift)
            
            # Generate augmented filename
            aug_name = f"{orig_path.stem}_aug{i+1}.wav"
            aug_path = output_path / aug_name
            
            # Save augmented audio
            sf.write(str(aug_path), y_aug, sr)
            
            # Add augmented file info
            augmented_info.append({
                'segment_path': str(aug_path),
                'instrument_label': row['instrument_label'],
                'participant_id': row['participant_id'],
                'dataset': row['dataset'],
                'original_wav': row['original_wav'],
                'onset_time': row['onset_time']
            })
    
    # Create and save augmented segment info
    augmented_df = pd.DataFrame(augmented_info)
    augmented_df.to_csv('EDA/segment_info_augmented.csv', index=False)
    
    # Print summary
    total_files = len(augmented_info)
    original_count = len(original_info)
    print(f"\nAugmentation complete!")
    print(f"Original segments: {original_count}")
    print(f"Total segments after augmentation: {total_files}")
    print(f"New segments added: {total_files - original_count}")
    
    return output_path

In [23]:
augmented_dir = augment_segments("segments", "augmentedSegments", num_augmentations=5)

Copying original segments...


5714it [00:02, 2126.78it/s]



Generating augmented segments...


5714it [01:44, 54.78it/s]


Augmentation complete!
Original segments: 5714
Total segments after augmentation: 34284
New segments added: 28570


# Extract MFCC Features

In [2]:
def extract_mfcc_features(segment_info_path, segments_dir='segments', n_mfcc=13):
    """
    Extract MFCC features from segmented audio files.
    
    Args:
        segment_info_path: Path to segment_info.csv
        segments_dir: Directory containing the audio segments (either 'segments' or 'augmentedSegments')
        n_mfcc: Number of MFCC coefficients to compute
    """
    # Load segment info
    metadata = pd.read_csv(segment_info_path)
    
    # Update paths to use the specified segments directory
    segments_path = Path(segments_dir)
    metadata['segment_path'] = metadata['segment_path'].apply(
        lambda x: str(segments_path / Path(x).name))
    
    # Initialize arrays to store features and labels
    features = []
    labels = []
    
    print(f"Extracting MFCC features from {segments_dir}...")
    for idx, row in tqdm(metadata.iterrows(), total=len(metadata)):
        try:
            # Load audio segment
            y, sr = librosa.load(row['segment_path'])
            
            # Extract MFCCs
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
            
            # Take mean of each coefficient over time
            mfcc_mean = np.mean(mfcc, axis=1)
            
            features.append(mfcc_mean)
            labels.append(row['instrument_label'])
            
        except Exception as e:
            print(f"Error processing {row['segment_path']}: {str(e)}")
            metadata = metadata.drop(idx)
            continue
    
    # Convert to numpy arrays
    X = np.array(features)
    y = np.array(labels)
    
    # Create features directory if it doesn't exist
    output_dir = Path('features')
    output_dir.mkdir(exist_ok=True, parents=True)
    
    # Save features and labels with directory-specific names
    dir_suffix = '_augmented' if segments_dir == 'augmentedSegments' else ''
    np.save(output_dir / f'mfcc_features{dir_suffix}.npy', X)
    np.save(output_dir / f'labels{dir_suffix}.npy', y)
    
    return X, y, metadata

In [25]:
segment_info_path = 'EDA/segment_info.csv'
augmented_segment_info_path = 'EDA/segment_info_augmented.csv'
    
    
X_orig, y_orig, metadata_orig = extract_mfcc_features(
    segment_info_path,
    segments_dir='segments'
)

X_aug, y_aug, metadata_aug = extract_mfcc_features(
    augmented_segment_info_path,
    segments_dir='augmentedSegments'
)

    
# # Show sample of the features
# print("\nSample MFCC features (first 3 segments):")
# print(X[:3])

Extracting MFCC features from segments...


100%|██████████| 5714/5714 [00:08<00:00, 704.89it/s]


Extracting MFCC features from augmentedSegments...


100%|██████████| 34284/34284 [00:47<00:00, 723.93it/s]


# K-Means Unsupervised Learning

In [30]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA

def cluster_sounds(n_clusters=4, features_path='features'):
    """
    Perform K-means clustering on MFCC features
    """
    # Load the MFCC features
    # features_path = Path('../projectFiles/features')
    # X = np.load(features_path / 'mfcc_features.npy')
    
    X = np.load(features_path)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform K-means clustering
    print(f"Performing K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # Save cluster assignments
    if features_path == 'features/mfcc_features_augmented.npy':
        np.save('features/cluster_labels_augmented.npy', cluster_labels)
    else:
        np.save('features/cluster_labels.npy', cluster_labels)
    
    # Print cluster sizes
    print("\nCluster sizes:")
    for i in range(n_clusters):
        print(f"Cluster {i}: {np.sum(cluster_labels == i)} sounds")
    
    return cluster_labels, kmeans

In [58]:
# First run clustering
cluster_labels_aug, kmeans_modelaug = cluster_sounds(n_clusters=4, features_path='features/mfcc_features_augmented.npy')

cluster_labels_orig, kmeans_model_orig = cluster_sounds(n_clusters=4, features_path='features/mfcc_features.npy')

Performing K-means clustering with 4 clusters...

Cluster sizes:
Cluster 0: 9371 sounds
Cluster 1: 5474 sounds
Cluster 2: 11101 sounds
Cluster 3: 8338 sounds
Performing K-means clustering with 4 clusters...

Cluster sizes:
Cluster 0: 1026 sounds
Cluster 1: 1522 sounds
Cluster 2: 1992 sounds
Cluster 3: 1174 sounds


# Evaluating K-Means

In [55]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import seaborn as sns

def evaluate_clustering(features_path='features', labels_path='labels.npy', cluster_labels_path='cluster_labels.npy'):
    """
    Evaluate clustering results using both internal and external metrics
    """
    # Load the data
    # features_path = Path('../projectFiles/features')
    X = np.load(features_path)
    y_true = np.load(labels_path)
    cluster_labels = np.load(cluster_labels_path)
    
    # Standardize features (same as in clustering)
    X_scaled = StandardScaler().fit_transform(X)
    
    # 1. Internal Metrics
    silhouette = silhouette_score(X_scaled, cluster_labels)
    davies_bouldin = davies_bouldin_score(X_scaled, cluster_labels)
    calinski_harabasz = calinski_harabasz_score(X_scaled, cluster_labels)
    
    print("Internal Metrics:")
    print(f"Silhouette Score: {silhouette:.3f} (ranges from -1 to 1, higher is better)")
    print(f"Davies-Bouldin Index: {davies_bouldin:.3f} (lower is better)")
    print(f"Calinski-Harabasz Index: {calinski_harabasz:.3f} (higher is better)")
    
    # 2. External Metrics
    ari = adjusted_rand_score(y_true, cluster_labels)
    nmi = normalized_mutual_info_score(y_true, cluster_labels)
    
    print("\nExternal Metrics:")
    print(f"Adjusted Rand Index: {ari:.3f} (ranges from -1 to 1, higher is better)")
    print(f"Normalized Mutual Information: {nmi:.3f} (ranges from 0 to 1, higher is better)")
    
    # 3. Create confusion matrix
    unique_labels = np.unique(y_true)
    unique_clusters = np.unique(cluster_labels)
    confusion_matrix = np.zeros((len(unique_labels), len(unique_clusters)))
    
    for i, label in enumerate(unique_labels):
        for j, cluster in enumerate(unique_clusters):
            confusion_matrix[i, j] = np.sum((y_true == label) & (cluster_labels == cluster))
    
    # Normalize by row (true labels)
    confusion_matrix_normalized = confusion_matrix / confusion_matrix.sum(axis=1)[:, np.newaxis]
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_matrix_normalized, 
                annot=True, 
                fmt='.2f', 
                xticklabels=[f'Cluster {i}' for i in range(len(unique_clusters))],
                yticklabels=unique_labels,
                cmap='YlOrRd')
    plt.title('Normalized Confusion Matrix:\nTrue Labels vs Cluster Assignments')
    plt.xlabel('Predicted Cluster')
    plt.ylabel('True Label')
    
    # Save confusion matrix
    viz_dir = Path(features_path).parent.parent / 'visualization'
    viz_dir.mkdir(exist_ok=True, parents=True)
    if features_path == 'features/mfcc_features_augmented.npy':
        plt.savefig(viz_dir / 'confusion_matrix_augmented.png', dpi=300, bbox_inches='tight')
    else:
        plt.savefig(viz_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 4. Print cluster composition
    print("\nCluster Composition:")
    for cluster in unique_clusters:
        cluster_mask = cluster_labels == cluster
        print(f"\nCluster {cluster}:")
        for label in unique_labels:
            count = np.sum((y_true == label) & cluster_mask)
            percentage = (count / np.sum(cluster_mask)) * 100
            print(f"{label}: {count} samples ({percentage:.1f}%)")

    return {
        'silhouette': silhouette,
        'davies_bouldin': davies_bouldin,
        'calinski_harabasz': calinski_harabasz,
        'ari': ari,
        'nmi': nmi,
        'confusion_matrix': confusion_matrix,
        'confusion_matrix_normalized': confusion_matrix_normalized
    }

In [56]:
evaluation_results_orig = evaluate_clustering(features_path='features/mfcc_features.npy', labels_path='features/labels.npy', cluster_labels_path='features/cluster_labels.npy')

Internal Metrics:
Silhouette Score: 0.120 (ranges from -1 to 1, higher is better)
Davies-Bouldin Index: 2.136 (lower is better)
Calinski-Harabasz Index: 611.699 (higher is better)

External Metrics:
Adjusted Rand Index: 0.108 (ranges from -1 to 1, higher is better)
Normalized Mutual Information: 0.115 (ranges from 0 to 1, higher is better)

Cluster Composition:

Cluster 0:
hhc: 174 samples (17.0%)
hho: 312 samples (30.4%)
kd: 125 samples (12.2%)
sd: 415 samples (40.4%)

Cluster 1:
hhc: 288 samples (18.9%)
hho: 115 samples (7.6%)
kd: 951 samples (62.5%)
sd: 168 samples (11.0%)

Cluster 2:
hhc: 519 samples (26.1%)
hho: 184 samples (9.2%)
kd: 596 samples (29.9%)
sd: 693 samples (34.8%)

Cluster 3:
hhc: 517 samples (44.0%)
hho: 398 samples (33.9%)
kd: 104 samples (8.9%)
sd: 155 samples (13.2%)


In [57]:
evaluation_results_aug = evaluate_clustering(features_path='features/mfcc_features_augmented.npy', labels_path='features/labels_augmented.npy', cluster_labels_path='features/cluster_labels_augmented.npy')

Internal Metrics:
Silhouette Score: 0.118 (ranges from -1 to 1, higher is better)
Davies-Bouldin Index: 2.219 (lower is better)
Calinski-Harabasz Index: 3998.029 (higher is better)

External Metrics:
Adjusted Rand Index: 0.076 (ranges from -1 to 1, higher is better)
Normalized Mutual Information: 0.076 (ranges from 0 to 1, higher is better)

Cluster Composition:

Cluster 0:
hhc: 3199 samples (34.1%)
hho: 2739 samples (29.2%)
kd: 1133 samples (12.1%)
sd: 2300 samples (24.5%)

Cluster 1:
hhc: 1704 samples (31.1%)
hho: 1342 samples (24.5%)
kd: 1078 samples (19.7%)
sd: 1350 samples (24.7%)

Cluster 2:
hhc: 2670 samples (24.1%)
hho: 1238 samples (11.2%)
kd: 3193 samples (28.8%)
sd: 4000 samples (36.0%)

Cluster 3:
hhc: 1415 samples (17.0%)
hho: 735 samples (8.8%)
kd: 5252 samples (63.0%)
sd: 936 samples (11.2%)


# Plotting Clustering Results from K-Means

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path

def plot_3d_visualization():
    """
    Create 3D visualizations of both true labels and cluster assignments
    """
    # Load the data
    features_path = Path('../projectFiles/features')
    X = np.load(features_path / 'mfcc_features.npy')
    y = np.load(features_path / 'labels.npy')
    cluster_labels = np.load(features_path / 'cluster_labels.npy')
    
    # Standardize features (same as in clustering)
    X_scaled = StandardScaler().fit_transform(X)
    
    # Reduce to 3 dimensions using PCA
    pca = PCA(n_components=3)
    X_pca = pca.fit_transform(X_scaled)
    
    # Create visualization directories
    viz_dir = features_path.parent / 'visualization'
    viz_3d_dir = viz_dir / '3d'
    viz_3d_clusters = viz_3d_dir / 'clusters'
    viz_3d_true = viz_3d_dir / 'true'
    
    # Create all directories
    for dir_path in [viz_dir, viz_3d_dir, viz_3d_clusters, viz_3d_true]:
        dir_path.mkdir(exist_ok=True, parents=True)
    
    # Plot 1: True Labels
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot each instrument type
    for label in np.unique(y):
        mask = y == label
        ax.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
                  label=label, alpha=0.6)
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
    ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%})')
    ax.set_title('Drum Sounds with True Labels (3D PCA)')
    ax.legend()
    
    # Save with multiple angles
    for angle in [0, 45, 90, 135]:
        ax.view_init(elev=20, azim=angle)
        plt.savefig(viz_3d_true / f'true_labels_angle_{angle}.png', 
                   dpi=300, bbox_inches='tight')
    
    plt.close()
    
    # Plot 2: Cluster Assignments
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                        c=cluster_labels, cmap='viridis', alpha=0.6)
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
    ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%})')
    ax.set_title('Drum Sound Clusters (3D PCA)')
    
    # Add colorbar
    plt.colorbar(scatter, label='Cluster')
    
    # Save with multiple angles
    for angle in [0, 45, 90, 135]:
        ax.view_init(elev=20, azim=angle)
        plt.savefig(viz_3d_clusters / f'clusters_angle_{angle}.png', 
                   dpi=300, bbox_inches='tight')
    
    plt.close()
    
    # Print explained variance
    total_var = sum(pca.explained_variance_ratio_[:3])
    print(f"\nTotal variance explained by first 3 PCs: {total_var:.1%}")
    print("Individual contributions:")
    for i, var in enumerate(pca.explained_variance_ratio_[:3]):
        print(f"PC{i+1}: {var:.1%}")


In [ ]:
def plot_2d_visualization():
    """
    Create 2D visualizations of both true labels and cluster assignments
    """
    # Load the data
    features_path = Path('../projectFiles/features')
    X = np.load(features_path / 'mfcc_features.npy')
    cluster_labels = np.load(features_path / 'cluster_labels.npy')
    
    # Standardize features (same as in clustering)
    X_scaled = StandardScaler().fit_transform(X)
    
    # Use PCA to reduce to 2 dimensions for visualization
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Create visualization directories
    viz_dir = features_path.parent / 'visualization'
    viz_2d_dir = viz_dir / '2d'
    viz_2d_clusters = viz_2d_dir / 'clusters'
    viz_2d_true = viz_2d_dir / 'true'
    
    # Create all directories
    for dir_path in [viz_dir, viz_2d_dir, viz_2d_clusters, viz_2d_true]:
        dir_path.mkdir(exist_ok=True, parents=True)
    
    # Create scatter plot for clusters
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                         c=cluster_labels, 
                         cmap='viridis', 
                         alpha=0.6)
    plt.title('Drum Sound Clusters (PCA Visualization)')
    plt.xlabel(f'First Principal Component\nExplained Variance: {pca.explained_variance_ratio_[0]:.2%}')
    plt.ylabel(f'Second Principal Component\nExplained Variance: {pca.explained_variance_ratio_[1]:.2%}')
    plt.colorbar(scatter, label='Cluster')
    
    # Add cluster centers if kmeans model is available
    try:
        kmeans = KMeans(n_clusters=len(np.unique(cluster_labels)), random_state=42)
        kmeans.fit(X_scaled)
        centers_pca = pca.transform(kmeans.cluster_centers_)
        plt.scatter(centers_pca[:, 0], centers_pca[:, 1], 
                   c='red', marker='x', s=200, linewidths=3, 
                   label='Cluster Centers')
        plt.legend()
    except Exception as e:
        print(f"Could not plot cluster centers: {e}")
    
    # Save the cluster plot
    plt.savefig(viz_2d_clusters / 'cluster_visualization.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Optional: Plot true labels if available
    try:
        y = np.load(features_path / 'labels.npy')
        
        # Create a second plot with true labels
        plt.figure(figsize=(12, 8))
        for label in np.unique(y):
            mask = y == label
            plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                       label=label, alpha=0.6)
        plt.title('Drum Sounds with True Labels (PCA Visualization)')
        plt.xlabel(f'First Principal Component\nExplained Variance: {pca.explained_variance_ratio_[0]:.2%}')
        plt.ylabel(f'Second Principal Component\nExplained Variance: {pca.explained_variance_ratio_[1]:.2%}')
        plt.legend()
        plt.savefig(viz_2d_true / 'true_labels_visualization.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    except FileNotFoundError:
        print("No labels file found - skipping true label visualization")
    
    # Print explained variance
    print(f"\nTotal variance explained by first 2 PCs: {sum(pca.explained_variance_ratio_[:2]):.1%}")
    print("Individual contributions:")
    for i, var in enumerate(pca.explained_variance_ratio_[:2]):
        print(f"PC{i+1}: {var:.1%}")